In [ ]:
!python3 -m pip install kagglehub





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 428.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 kB 229.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 3.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip3 install --upgrade pip


NameError: name 'kagglehub' is not defined

In [1]:
import kagglehub
path = kagglehub.dataset_download("andrewmvd/road-sign-detection")
print("Path to dataset files:", path)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 218M/218M [06:49<00:00, 559kB/s] 

Extracting files...


Path to dataset files: /Users/admin/.cache/kagglehub/datasets/andrewmvd/road-sign-detection/versions/1


In [2]:
import os

path = "/Users/admin/.cache/kagglehub/datasets/andrewmvd/road-sign-detection/versions/1"

print(os.listdir(path))


['images', 'annotations']


In [3]:
import os
import xml.etree.ElementTree as ET
from collections import Counter

BASE_PATH = "/Users/admin/.cache/kagglehub/datasets/andrewmvd/road-sign-detection/versions/1"
ANNOTATIONS_PATH = os.path.join(BASE_PATH, "annotations")

def generate_caption_from_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    labels = []

    for obj in root.findall("object"):
        label = obj.find("name").text
        labels.append(label)

    if not labels:
        return "No road signs detected."

    counts = Counter(labels)

    phrases = []
    for label, count in counts.items():
        label = label.lower()
        if count == 1:
            phrases.append(f"a {label}")
        else:
            phrases.append(f"{count} {label}s")

    if len(phrases) == 1:
        return f"There is {phrases[0]} in the image."
    else:
        return "There are " + ", ".join(phrases[:-1]) + " and " + phrases[-1] + " in the image."


captions = []
image_ids = []

for file in os.listdir(ANNOTATIONS_PATH):
    if file.endswith(".xml"):
        xml_path = os.path.join(ANNOTATIONS_PATH, file)
        caption = generate_caption_from_xml(xml_path)

        image_name = file.replace(".xml", ".jpg")
        captions.append(caption)
        image_ids.append(image_name)

print("Number of captions:", len(captions))
print("Example captions:")
print(captions[:5])


Number of captions: 877
Example captions:
['There is 2 speedlimits in the image.', 'There is 2 speedlimits in the image.', 'There are a stop and a trafficlight in the image.', 'There is a speedlimit in the image.', 'There is a speedlimit in the image.']


In [4]:
LABEL_MAP = {
    "stop": "stop sign",
    "trafficlight": "traffic light",
    "traffic light": "traffic light",
    "speedlimit": "speed limit sign",
    "crosswalk": "crosswalk sign"
}

def generate_caption_from_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    labels = []

    for obj in root.findall("object"):
        raw_label = obj.find("name").text.lower()
        label = LABEL_MAP.get(raw_label, raw_label)
        labels.append(label)

    if not labels:
        return "No road signs detected."

    counts = Counter(labels)
    phrases = []

    for label, count in counts.items():
        if count == 1:
            phrases.append(f"a {label}")
        else:
            phrases.append(f"{count} {label}s")

    if len(phrases) == 1:
        if count == 1:
            return f"There is {phrases[0]} in the image."
        else:
            return f"There are {phrases[0]} in the image."
    else:
        return "There are " + ", ".join(phrases[:-1]) + " and " + phrases[-1] + " in the image."


In [5]:
def generate_short_caption(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    labels = []

    for obj in root.findall("object"):
        raw_label = obj.find("name").text.lower()
        label = LABEL_MAP.get(raw_label, raw_label)
        labels.append(label)

    if not labels:
        return "No road signs detected."

    counts = Counter(labels)
    phrases = []

    for label, count in counts.items():
        if count == 1:
            phrases.append(f"a {label}")
        else:
            phrases.append(f"{count} {label}s")

    if len(phrases) == 1:
        if list(counts.values())[0] == 1:
            return f"There is {phrases[0]} in the image."
        else:
            return f"There are {phrases[0]} in the image."
    else:
        return "There are " + ", ".join(phrases[:-1]) + " and " + phrases[-1] + " in the image."


In [6]:
LONG_DESCRIPTIONS = {
    "stop sign": "A stop sign is a regulatory traffic sign that instructs drivers to come to a complete stop at an intersection to ensure road safety.",
    "traffic light": "A traffic light is a signaling device positioned at road intersections to control vehicle and pedestrian movement using colored lights.",
    "speed limit sign": "A speed limit sign indicates the maximum legal speed allowed for vehicles on a specific road segment.",
    "crosswalk sign": "A crosswalk sign warns drivers about pedestrian crossing areas and helps improve pedestrian safety."
}


In [7]:
def generate_long_caption(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    labels = []

    for obj in root.findall("object"):
        raw_label = obj.find("name").text.lower()
        label = LABEL_MAP.get(raw_label, raw_label)
        labels.append(label)

    if not labels:
        return "The image does not contain any visible road signs."

    counts = Counter(labels)
    explanations = []

    for label, count in counts.items():
        description = LONG_DESCRIPTIONS.get(label, "")
        if count == 1:
            explanations.append(description)
        else:
            explanations.append(f"There are {count} instances of {label}s in the image. {description}")

    return " ".join(explanations)


In [8]:
short_captions = []
long_captions = []

for file in os.listdir(ANNOTATIONS_PATH):
    if file.endswith(".xml"):
        xml_path = os.path.join(ANNOTATIONS_PATH, file)

        short_cap = generate_short_caption(xml_path)
        long_cap = generate_long_caption(xml_path)

        short_captions.append(short_cap)
        long_captions.append(long_cap)

print("Number of short captions:", len(short_captions))
print("Example short:", short_captions[:3])
print("\nExample long:", long_captions[:2])


Number of short captions: 877
Example short: ['There are 2 speed limit signs in the image.', 'There are 2 speed limit signs in the image.', 'There are a stop sign and a traffic light in the image.']

Example long: ['There are 2 instances of speed limit signs in the image. A speed limit sign indicates the maximum legal speed allowed for vehicles on a specific road segment.', 'There are 2 instances of speed limit signs in the image. A speed limit sign indicates the maximum legal speed allowed for vehicles on a specific road segment.']


In [9]:
all_sentences = short_captions + long_captions

print("Total sentences:", len(all_sentences))
print(all_sentences[:3])


Total sentences: 1754
['There are 2 speed limit signs in the image.', 'There are 2 speed limit signs in the image.', 'There are a stop sign and a traffic light in the image.']


In [10]:
from collections import Counter

def build_vocab(sentences, min_freq=1):
    counter = Counter()

    for sentence in sentences:
        tokens = sentence.lower().split()
        counter.update(tokens)

    vocab = {
        "<PAD>": 0,
        "<SOS>": 1,
        "<EOS>": 2,
        "<UNK>": 3
    }

    idx = 4
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = idx
            idx += 1

    return vocab

vocab = build_vocab(all_sentences)
print("Vocabulary size:", len(vocab))


Vocabulary size: 69


In [11]:
import torch
from torch.utils.data import Dataset

class CaptionDataset(Dataset):
    def __init__(self, sentences, vocab, max_len=40):
        self.sentences = sentences
        self.vocab = vocab
        self.max_len = max_len

    def encode(self, sentence):
        tokens = sentence.lower().split()

        encoded = [self.vocab["<SOS>"]]

        for token in tokens:
            encoded.append(self.vocab.get(token, self.vocab["<UNK>"]))

        encoded.append(self.vocab["<EOS>"])

        if len(encoded) < self.max_len:
            encoded += [self.vocab["<PAD>"]] * (self.max_len - len(encoded))
        else:
            encoded = encoded[:self.max_len]

        return torch.tensor(encoded)

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        encoded = self.encode(self.sentences[idx])
        return encoded[:-1], encoded[1:]

dataset = CaptionDataset(all_sentences, vocab)


In [12]:
import torch.nn as nn

class CaptionLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embed = self.embedding(x)
        out, _ = self.lstm(embed)
        out = self.fc(out)
        return out


In [13]:
from torch.utils.data import DataLoader
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CaptionLSTM(len(vocab)).to(device)

loader = DataLoader(dataset, batch_size=32, shuffle=True)

criterion = nn.CrossEntropyLoss(ignore_index=vocab["<PAD>"])
optimizer = optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 15

for epoch in range(EPOCHS):
    total_loss = 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        output = model(x)

        loss = criterion(
            output.reshape(-1, len(vocab)),
            y.reshape(-1)
        )

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")


Epoch 1, Loss: 1.7918
Epoch 2, Loss: 0.2819
Epoch 3, Loss: 0.1961
Epoch 4, Loss: 0.1817
Epoch 5, Loss: 0.1772
Epoch 6, Loss: 0.1739
Epoch 7, Loss: 0.1710
Epoch 8, Loss: 0.1698
Epoch 9, Loss: 0.1689
Epoch 10, Loss: 0.1674
Epoch 11, Loss: 0.1675
Epoch 12, Loss: 0.1656
Epoch 13, Loss: 0.1655
Epoch 14, Loss: 0.1648
Epoch 15, Loss: 0.1653


In [15]:
idx2word = {v:k for k,v in vocab.items()}

def generate_text(model, vocab, max_len=40):
    model.eval()

    input_seq = torch.tensor([[vocab["<SOS>"]]]).to(device)
    generated = []

    for _ in range(max_len):
        output = model(input_seq)
        next_token = output[:, -1, :].argmax(1).item()

        if next_token == vocab["<EOS>"]:
            break

        generated.append(idx2word.get(next_token, "<UNK>"))

        input_seq = torch.cat(
            [input_seq, torch.tensor([[next_token]]).to(device)],
            dim=1
        )

    return " ".join(generated)

print(generate_text(model, vocab))



there is a speed limit sign in the image.


In [16]:
for _ in range(5):
    print(generate_text(model, vocab))

there is a speed limit sign in the image.
there is a speed limit sign in the image.
there is a speed limit sign in the image.
there is a speed limit sign in the image.
there is a speed limit sign in the image.


In [17]:
training_inputs = []
training_targets = []

for sentence in short_captions:
    clean = sentence.lower()
    
    # Extract label phrase
    label_part = clean.replace("there is ", "") \
                      .replace("there are ", "") \
                      .replace(" in the image.", "")
    
    training_inputs.append(label_part)
    training_targets.append(clean)

print(training_inputs[:3])
print(training_targets[:3])


['2 speed limit signs', '2 speed limit signs', 'a stop sign and a traffic light']
['there are 2 speed limit signs in the image.', 'there are 2 speed limit signs in the image.', 'there are a stop sign and a traffic light in the image.']


In [18]:
from collections import Counter

def build_vocab(sentences, min_freq=1):
    counter = Counter()

    for sentence in sentences:
        tokens = sentence.split()
        counter.update(tokens)

    vocab = {
        "<PAD>": 0,
        "<SOS>": 1,
        "<EOS>": 2,
        "<UNK>": 3
    }

    idx = 4
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = idx
            idx += 1

    return vocab

all_text = training_inputs + training_targets
vocab = build_vocab(all_text)

print("Vocabulary size:", len(vocab))


Vocabulary size: 28


In [19]:
import torch
from torch.utils.data import Dataset

class ConditionalCaptionDataset(Dataset):
    def __init__(self, inputs, targets, vocab, max_len=30):
        self.inputs = inputs
        self.targets = targets
        self.vocab = vocab
        self.max_len = max_len

    def encode(self, sentence):
        tokens = sentence.split()
        encoded = [self.vocab["<SOS>"]]

        for token in tokens:
            encoded.append(self.vocab.get(token, self.vocab["<UNK>"]))

        encoded.append(self.vocab["<EOS>"])

        if len(encoded) < self.max_len:
            encoded += [self.vocab["<PAD>"]] * (self.max_len - len(encoded))
        else:
            encoded = encoded[:self.max_len]

        return torch.tensor(encoded)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_encoded = self.encode(self.inputs[idx])
        target_encoded = self.encode(self.targets[idx])

        return input_encoded, target_encoded

dataset = ConditionalCaptionDataset(training_inputs, training_targets, vocab)


In [20]:
import torch.nn as nn

class Seq2SeqCaptionModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)

        self.encoder = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(embed_dim, hidden_dim, batch_first=True)

        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_seq, target_seq):
        # Encode input
        embedded_input = self.embedding(input_seq)
        _, (hidden, cell) = self.encoder(embedded_input)

        # Decode target
        embedded_target = self.embedding(target_seq)
        outputs, _ = self.decoder(embedded_target, (hidden, cell))

        outputs = self.fc(outputs)

        return outputs


In [21]:
from torch.utils.data import DataLoader
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Seq2SeqCaptionModel(len(vocab)).to(device)

loader = DataLoader(dataset, batch_size=32, shuffle=True)

criterion = nn.CrossEntropyLoss(ignore_index=vocab["<PAD>"])
optimizer = optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 20

for epoch in range(EPOCHS):
    total_loss = 0

    for input_seq, target_seq in loader:
        input_seq = input_seq.to(device)
        target_seq = target_seq.to(device)

        optimizer.zero_grad()

        output = model(input_seq, target_seq[:, :-1])

        loss = criterion(
            output.reshape(-1, len(vocab)),
            target_seq[:, 1:].reshape(-1)
        )

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")


Epoch 1, Loss: 1.4784
Epoch 2, Loss: 0.3632
Epoch 3, Loss: 0.2553
Epoch 4, Loss: 0.2217
Epoch 5, Loss: 0.2098
Epoch 6, Loss: 0.2034
Epoch 7, Loss: 0.1822
Epoch 8, Loss: 0.1380
Epoch 9, Loss: 0.1134
Epoch 10, Loss: 0.0799
Epoch 11, Loss: 0.0668
Epoch 12, Loss: 0.0510
Epoch 13, Loss: 0.0406
Epoch 14, Loss: 0.0357
Epoch 15, Loss: 0.0284
Epoch 16, Loss: 0.0241
Epoch 17, Loss: 0.0224
Epoch 18, Loss: 0.0177
Epoch 19, Loss: 0.0165
Epoch 20, Loss: 0.0138


In [22]:
idx2word = {v:k for k,v in vocab.items()}

def generate_caption(model, vocab, input_text, max_len=30):
    model.eval()

    tokens = input_text.split()
    encoded_input = [vocab["<SOS>"]]

    for token in tokens:
        encoded_input.append(vocab.get(token, vocab["<UNK>"]))

    encoded_input.append(vocab["<EOS>"])

    input_tensor = torch.tensor([encoded_input]).to(device)

    with torch.no_grad():
        embedded_input = model.embedding(input_tensor)
        _, (hidden, cell) = model.encoder(embedded_input)

    decoder_input = torch.tensor([[vocab["<SOS>"]]]).to(device)

    generated = []

    for _ in range(max_len):
        embedded = model.embedding(decoder_input)
        output, (hidden, cell) = model.decoder(embedded, (hidden, cell))
        prediction = model.fc(output[:, -1, :])

        next_token = prediction.argmax(1).item()

        if next_token == vocab["<EOS>"]:
            break

        generated.append(idx2word.get(next_token, "<UNK>"))

        decoder_input = torch.tensor([[next_token]]).to(device)

    return " ".join(generated)


In [23]:
print(generate_caption(model, vocab, "a stop sign"))
print(generate_caption(model, vocab, "2 speed limit signs"))
print(generate_caption(model, vocab, "a traffic light"))


there is a crosswalk sign in the image.
speed limit sign in the image.
there are a traffic light in the image.


In [24]:
def generate_caption_beam(model, vocab, input_text, beam_width=3, max_len=30):
    model.eval()
    idx2word = {v:k for k,v in vocab.items()}

    tokens = input_text.split()
    encoded_input = [vocab["<SOS>"]]
    for token in tokens:
        encoded_input.append(vocab.get(token, vocab["<UNK>"]))
    encoded_input.append(vocab["<EOS>"])

    input_tensor = torch.tensor([encoded_input]).to(device)

    with torch.no_grad():
        embedded_input = model.embedding(input_tensor)
        _, (hidden, cell) = model.encoder(embedded_input)

    sequences = [[ [vocab["<SOS>"]], 0.0, hidden, cell ]]

    for _ in range(max_len):
        all_candidates = []

        for seq, score, h, c in sequences:
            decoder_input = torch.tensor([[seq[-1]]]).to(device)
            embedded = model.embedding(decoder_input)
            output, (h_new, c_new) = model.decoder(embedded, (h, c))
            prediction = model.fc(output[:, -1, :])

            log_probs = torch.log_softmax(prediction, dim=1)
            topk = torch.topk(log_probs, beam_width)

            for i in range(beam_width):
                token = topk.indices[0][i].item()
                candidate = [
                    seq + [token],
                    score - topk.values[0][i].item(),
                    h_new,
                    c_new
                ]
                all_candidates.append(candidate)

        sequences = sorted(all_candidates, key=lambda x: x[1])[:beam_width]

    best_seq = sequences[0][0]

    words = []
    for token in best_seq:
        if token == vocab["<EOS>"]:
            break
        if token not in [vocab["<SOS>"], vocab["<PAD>"]]:
            words.append(idx2word.get(token, ""))

    return " ".join(words)


In [25]:
print(generate_caption_beam(model, vocab, "a stop sign"))
print(generate_caption_beam(model, vocab, "2 speed limit signs"))
print(generate_caption_beam(model, vocab, "a traffic light"))


there is a crosswalk sign in the image.
speed limit sign in the image.
there is a traffic light in the image.


In [26]:
def generate_caption_stable(model, vocab, input_text, max_len=30):
    model.eval()
    idx2word = {v:k for k,v in vocab.items()}

    tokens = input_text.split()
    encoded_input = [vocab["<SOS>"]]

    for token in tokens:
        encoded_input.append(vocab.get(token, vocab["<UNK>"]))

    encoded_input.append(vocab["<EOS>"])
    input_tensor = torch.tensor([encoded_input]).to(device)

    with torch.no_grad():
        embedded_input = model.embedding(input_tensor)
        _, (hidden, cell) = model.encoder(embedded_input)

    decoder_input = torch.tensor([[vocab["<SOS>"]]]).to(device)
    generated = []

    for _ in range(max_len):
        embedded = model.embedding(decoder_input)
        output, (hidden, cell) = model.decoder(embedded, (hidden, cell))
        prediction = model.fc(output[:, -1, :])

        next_token = prediction.argmax(1).item()

        if next_token == vocab["<EOS>"]:
            break

        word = idx2word.get(next_token, "")
        generated.append(word)

        decoder_input = torch.tensor([[next_token]]).to(device)

    sentence = " ".join(generated)

    # ---- Grammar Stabilization ----
    if input_text.strip().startswith("a "):
        if not sentence.startswith("there is"):
            sentence = "there is " + input_text + " in the image."
    elif input_text.strip()[0].isdigit():
        if not sentence.startswith("there are"):
            sentence = "there are " + input_text + " in the image."

    return sentence


In [27]:
print(generate_caption_stable(model, vocab, "a stop sign"))
print(generate_caption_stable(model, vocab, "2 speed limit signs"))
print(generate_caption_stable(model, vocab, "a traffic light"))


there is a crosswalk sign in the image.
there are 2 speed limit signs in the image.
there is a traffic light in the image.


In [28]:
def evaluate_exact_match(model, dataset, vocab):
    model.eval()
    correct = 0

    for input_text, target_text in zip(training_inputs, training_targets):
        generated = generate_caption_stable(model, vocab, input_text)
        if generated.strip() == target_text.strip():
            correct += 1

    accuracy = correct / len(training_inputs)
    return accuracy

accuracy = evaluate_exact_match(model, dataset, vocab)
print("Exact Match Accuracy:", accuracy)


Exact Match Accuracy: 0.7947548460661346


In [29]:
def token_accuracy(model, dataset, vocab):
    model.eval()
    total = 0
    correct = 0

    loader = DataLoader(dataset, batch_size=1)

    for input_seq, target_seq in loader:
        input_seq = input_seq.to(device)
        target_seq = target_seq.to(device)

        output = model(input_seq, target_seq[:, :-1])
        predictions = output.argmax(2)

        for i in range(target_seq.shape[1]-1):
            if target_seq[0, i+1] != vocab["<PAD>"]:
                total += 1
                if predictions[0, i] == target_seq[0, i+1]:
                    correct += 1

    return correct / total

print("Token Accuracy:", token_accuracy(model, dataset, vocab))


Token Accuracy: 0.9965266471290568


In [30]:
from sklearn.model_selection import train_test_split

train_inputs, val_inputs, train_targets, val_targets = train_test_split(
    training_inputs,
    training_targets,
    test_size=0.2,
    random_state=42
)


In [31]:
train_dataset = ConditionalCaptionDataset(train_inputs, train_targets, vocab)
val_dataset   = ConditionalCaptionDataset(val_inputs, val_targets, vocab)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32)


In [32]:
EPOCHS = 25

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for input_seq, target_seq in train_loader:
        input_seq = input_seq.to(device)
        target_seq = target_seq.to(device)

        optimizer.zero_grad()

        output = model(input_seq, target_seq[:, :-1])

        loss = criterion(
            output.reshape(-1, len(vocab)),
            target_seq[:, 1:].reshape(-1)
        )

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # ---- Validation ----
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for input_seq, target_seq in val_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            output = model(input_seq, target_seq[:, :-1])

            loss = criterion(
                output.reshape(-1, len(vocab)),
                target_seq[:, 1:].reshape(-1)
            )

            val_loss += loss.item()

    print(f"Epoch {epoch+1}")
    print(f"Train Loss: {train_loss/len(train_loader):.4f}")
    print(f"Val Loss:   {val_loss/len(val_loader):.4f}")
    print("-" * 30)


Epoch 1
Train Loss: 0.0125
Val Loss:   0.0119
------------------------------
Epoch 2
Train Loss: 0.0100
Val Loss:   0.0092
------------------------------
Epoch 3
Train Loss: 0.0087
Val Loss:   0.0109
------------------------------
Epoch 4
Train Loss: 0.0085
Val Loss:   0.0096
------------------------------
Epoch 5
Train Loss: 0.0070
Val Loss:   0.0086
------------------------------
Epoch 6
Train Loss: 0.0066
Val Loss:   0.0090
------------------------------
Epoch 7
Train Loss: 0.0066
Val Loss:   0.0080
------------------------------
Epoch 8
Train Loss: 0.0063
Val Loss:   0.0077
------------------------------
Epoch 9
Train Loss: 0.0114
Val Loss:   0.0194
------------------------------
Epoch 10
Train Loss: 0.0120
Val Loss:   0.0099
------------------------------
Epoch 11
Train Loss: 0.0072
Val Loss:   0.0087
------------------------------
Epoch 12
Train Loss: 0.0075
Val Loss:   0.0086
------------------------------
Epoch 13
Train Loss: 0.0058
Val Loss:   0.0101
--------------------------

In [33]:
torch.save({
    'model_state_dict': model.state_dict(),
    'vocab': vocab
}, "nlp_model.pth")
